## 17. Iterators, Generators, and Classic Coroutines

In [1]:
import re
import reprlib

In [2]:
RE_WORD = re.compile(r'\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

In [3]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [4]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [5]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [6]:
s[0]

'The'

In [7]:
s[5]

'Walrus'

In [8]:
s[-1]

'said'

## Why Sequences Are Iterable: The `iter` function

Whenever Python needs to iterate over an object x, it automatically calls `iter(x)`.

The `iter` built-in function:

1. Checks whether the object implements `__iter__`, and calls that to obtain an iterator.
2. If `__iter__` is not implemented, but `__getitem__` is, then `iter()` creates an iterator that tries to fetch items by index, starting from `0` (zero).
3. If that fails, Python raises `TypeError`, usually saying `'C' object is not iterable`, where `C` is the class of the target object.

In [10]:
class Spam:
    def __getitem__(self, i):
        print('->', i)
        raise IndexError()
    
spam_can = Spam()
iter(spam_can)

In [11]:
list(spam_can)

-> 0


[]

In [12]:
from collections import abc
isinstance(spam_can, abc.Iterable)

False

In [13]:
class GooseSpam:
    def __iter__(self):
        pass

from collections import abc
issubclass(GooseSpam, abc.Iterable)

True

In [14]:
goose_spam_can = GooseSpam()
isinstance(goose_spam_can, abc.Iterable)

True

### Using `iter` with a Callable

We can call `iter()` with two arguments to create an iterator from a function or any callable object. In this usage, the first argument must be a callable to be invoked repeatedly (with no arguments) to produce values, and the second argument is a _sentinel_: a marker value which, when returned by the callable, causes the iterator to raise StopIteration instead of yielding the sentinel.

In [28]:
from random import randint

def d6():
    return randint(1, 6)


In [29]:
d6_iter = iter(d6, 1)
d6_iter

In [31]:
for roll in d6_iter:
    print(roll)

## Iterables Versus Iterators

_iterable_
> Any object from which the `iter` built-in function can obtain an iterator. Objects implementing an `__iter__` method returning an iterator are iterable. Sequences are always iterable, as are objects implementing a `__getitem__` method that accepts 0-based indexes.

In [32]:
s = 'ABC'
for char in s:
    print(char)

A
B
C


## Sentence Classes with `__iter__`

The next variations of `Sentence` implement the standard iterable protocol, first by implementing the Iterator design pattern, and then with generator functions.


### Sentence Take #2: A Classic Iterator

In [33]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'
    
    def __iter__(self):
        return SentenceIterator(self.words)
    
class SentenceIterator:

    def __init__(self, words):
        self.words = words
        self.index = 0

    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index +=1
        return word
    
    def __iter__(self):
        return self

### Sentence Take #3: A Generator Function

In [34]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for word in self.words:
            yield word


### How a Generator Works

In [36]:
def gen_123():
    yield 1
    yield 2
    yield 3

In [37]:
gen_123

<function __main__.gen_123()>

In [38]:
gen_123()

<generator object gen_123 at 0x7fb6fc64b950>

In [40]:
for i in gen_123():
    print(i)

1
2
3


In [41]:
g = gen_123()
next(g)

1

In [42]:
next(g)

2

In [43]:
next(g)

3

In [44]:
next(g)

StopIteration: 

In [45]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')

In [46]:
for c in gen_AB():
    print('-->', c)

start
--> A
continue
--> B
end.


## Lazy Sentences

The final variations of `Sentence` are lazy, taking advantage of a lazy function from the `re` module.

### Sentence Take #4: Lazy Generator

The Iterator interface is designed to be lazy: `next(my_iterator)` yields one item at a time. The opposite of lazy is eager: lazy evaluation and eager evaluation are technical terms in programming language theory.

Our Sentence implementations so far have not been lazy because the `__init__` eagerly builds a list of all words in the text, binding it to the self.words attribute. This requires processing the entire text, and the list may use as much memory as the text itself (probably more; it depends on how many nonword characters are in the text). Most of this work will be in vain if the user only iterates over the first couple of words. If you wonder, “Is there a lazy way of doing this in Python?” the answer is often “Yes.”

The `re.finditer` function is a lazy version of `re.findall`. Instead of a list, `re.finditer` returns a generator yielding `re.MatchObject` instances on demand. If there are many matches, `re.finditer` saves a lot of memory. Using it, our third version of Sentence is now lazy: it only reads the next word from the text when it is needed. 

In [47]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:

    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'
    
    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()

In [48]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')

In [49]:
res1 = [x*3 for x in gen_AB()]

start
continue
end.


In [50]:
res1

['AAA', 'BBB']

In [51]:
for i in res1:
    print('-->', i)

--> AAA
--> BBB


In [52]:
res2 = (x*3 for x in gen_AB())
res2

<generator object <genexpr> at 0x7fb6e27292f0>

In [53]:
for i in res2:
    print('-->', i)

start
--> AAA
continue
--> BBB
end.


In [54]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:

    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'
    
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

## When to Use Generator Expressions

My rule of thumb in choosing the syntax to use is simple: if the generator expression spans more than a couple of lines, I prefer to code a generator function for the sake of readability.

## An Arithmetic Progression Generator

The classic Iterator pattern is all about traversal: navigating some data structure. But a standard interface based on a method to fetch the next item in a series is also useful when the items are produced on the fly, instead of retrieved from a collection. For example, the range built-in generates a bounded arithmetic progression (AP) of inte‐ gers. What if you need to generate an AP of numbers of any type, not only integers?

In [55]:
class ArithmeticProgression:

    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end # None -> "infinite series"

    def __iter__(self):
        result_type = type(self.begin + self.step)
        result = result_type(self.begin)
        forever = self.end is None
        index = 0
        while forever or result < self.end:
            yield result
            index += 1
            result = self.begin + self.step*index

In [56]:
ap = ArithmeticProgression(0, 1, 3)
list(ap)

[0, 1, 2]

In [57]:
ap = ArithmeticProgression(1, .5, 3)
list(ap)

[1.0, 1.5, 2.0, 2.5]

In [58]:
ap = ArithmeticProgression(0, 1/3, 1)
list(ap)

[0.0, 0.3333333333333333, 0.6666666666666666]

In [59]:
from fractions import Fraction
ap = ArithmeticProgression(0, Fraction(1, 3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [60]:
from decimal import Decimal
ap = ArithmeticProgression(0, Decimal('.1'), .3)
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

In [61]:
100 * 1.1

110.00000000000001

In [62]:
sum(1.1 for _ in range(100))

109.99999999999982

In [63]:
1000 * 1.1

1100.0

In [64]:
sum(1.1 for _ in range(1000))

1100.0000000000086

In [65]:
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step*index

In [66]:
ap = aritprog_gen(0, 1, 3)
list(ap)

[0, 1, 2]

In [67]:
ap = aritprog_gen(0, Decimal('.1'), .3)
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

## Generator Functions in the Standard Library

The standard library provides many generators, from plain-text file objects providing line-by-line iteration, to the awesome `os.walk` function, which yields filenames while traversing a directory tree, making recursive filesystem searches as simple as a `for` loop.

The `os.walk` generator function is impressive, but in this section I want to focus on general-purpose functions that take arbitrary iterables as arguments and return gen‐ erators that yield selected, computed, or rearranged items. In the following tables, I summarize two dozen of them, from the built-in, `itertools`, and `functools` modules. For convenience, I grouped them by high-level functionality, regardless of where they are defined.

The first group contains the filtering generator functions: they yield a subset of items produced by the input iterable, without changing the items themselves. Like `take while`, most functions listed in Table take a `predicate`, which is a one-argument Boolean function that will be applied to each item in the input to determine whether the item is included in the output.

| Module | Function | Description |
| :----- | :------- | :---------- |
| `itertools` | `compress(it, selector_it)` | Consumes two iterables in parallel; yields items from `it` whenever the corresponding item in `selector_it` is truthy |
| `itertools` | `dropwhile(predicate, it)` | Consumes `it`, skipping items while `predicate` computes truthy, then yields every remaining item (no further checks are made) |
| (built-in) | `filter(predicate, it)` | Applies `predicate` to each item of `iterable`, yielding the item if `predicate(item)` is truthy; if `predicate` is `None`, only truthy items are yielded |
| `itertools` | `filterfalse(predicate, it)` | Same as `filter`, with the `predicate` logic negated: yields items whenever `predicate` computes falsy |
| `itertools` | `islice(it, stop)` or `islice(it, start, stop, step=1)` | Yields items from a slice of `it`, similar to `s[:stop]` or `s[start:stop:step]` except `it` can be any iterable, and the operation is lazy | 
| `itertools` | `takewhile(predicate, it)` | Yields items while `predicate` computes truthy, then stops and no further checks are made |

In [69]:
def vowel(c):
    return c.lower() in 'aeiou'

In [70]:
list(
    filter(vowel, 'Aardvark')
)

['A', 'a', 'a']

In [71]:
import itertools

list(
    itertools.filterfalse(vowel, 'Aardvark')
)

['r', 'd', 'v', 'r', 'k']

In [72]:
list(
    itertools.dropwhile(vowel, 'Aardvark')
)

['r', 'd', 'v', 'a', 'r', 'k']

In [73]:
list(
    itertools.compress('Aardvark', (1,0,1,1,0,1))
)

['A', 'r', 'd', 'a']

In [74]:
list(
    itertools.islice('Aardvark', 4)
)

['A', 'a', 'r', 'd']

In [75]:
list(
    itertools.islice('Aardvark', 4, 7)
)

['v', 'a', 'r']

In [77]:
list(
    itertools.islice('Aardvark', 1, 7, 2)
)

['a', 'd', 'a']

The next group contains the mapping generators: they yield items computed from each individual item in the input iterable—or iterables, in the case of `map` and `starmap`. The generators in Table yield one result per item in the input iterables. If the input comes from more than one iterable, the output stops as soon as the first input iterable is exhausted.

| Module | Function | Description |
| :----- | :------- | :---------- |
| `itertools` | `accumulate(it, [func])` | Yields accumulated sums; if `func` is provided, yields the result of applying it to the first pair of items, then to the first result and next item, etc |
| (built-in) | `enumerate(iterable, start=0)` | Yields 2-tupples of the form `(index, item)`, where `index` is counted from `start`, and `item` is taken from the `iterable`. |
| (built-in) | `map(func, it1, [ it2, ..., itN ])` | Applies `func` to each item of `it`, yielding the result; if N iterables are given, `func`, must take N arguments and the iterables will be consumed in parallel | 
| `itertools` | `starmap(func, it)` | Applies `func` to each item of `it`, yielding the result; the input iterable should yield iterable items `iit`, and `funct` is applied as `func(*iit)` |

In [78]:
import itertools

sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
list(
    itertools.accumulate(sample)
)

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [79]:
list(
    itertools.accumulate(sample, min)
)

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [80]:
list(
    itertools.accumulate(sample, max)
)

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [81]:
import operator

list(
    itertools.accumulate(sample, operator.mul)
)

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [82]:
list(
    itertools.accumulate(range(1, 11), operator.mul)
)

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

In [83]:
list(
    map(operator.mul, range(11), range(11))
)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [84]:
list(
    map(operator.mul, range(11), [2, 4, 8])
)

[0, 4, 16]

In [85]:
list(
    map(lambda a, b: (a, b), range(11), [2, 4, 8])
)

[(0, 2), (1, 4), (2, 8)]

In [86]:
import itertools

list(
    itertools.starmap(operator.mul, enumerate('albatroz', 1))
)

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

In [87]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]

list(
    itertools.starmap(lambda a, b: b/a,
                      enumerate(itertools.accumulate(sample), 1)
                      )
)

[5.0,
 4.5,
 3.6666666666666665,
 4.75,
 5.2,
 5.333333333333333,
 5.0,
 4.375,
 4.888888888888889,
 4.5]

Next, we have the group of merging generators—all of these yield items from multiple input iterables. `chain` and `chain.from_iterable` consume the input iterables sequentially (one after the other), while `product`, `zip`, and `zip_longest` consume the input iterables in parallel. 

| Module | Function | Description |
| :----- | :------- | :---------- |
| `itertools` | `chain(it1, ..., itN)` | Yields all items from `it1`, then from `it2`,  etc., seamslessy |
| `itertools` | `chain.from_iterable(it)` | Yields all items from each iterable produced by `it`, one after the other, seamlessy; `it` will be an iterable where the items are also iterables, for example, a list of tuples | 
| `itertools` | `product(it1, ..., itN, repeat=1)` | Cartesian product: yields N-tuples made by combining items from each imput iterable, like nested `for` loops could produce; `repeat` allows the input iterables to be consumed more than once | 
| (built-in) | `zip(it1, ..., itN, strict=False)` | Yields N-tuples built from items taken from the iterables in parallel, silently stopping when the first iterable is exhausted, unless `strict=True` is given | 
| `itertools` | `zip_longest(it1, ..., itN, fillvalue=None)` | Yields N-tuples built from items taken from the iterables in parallel, stopping only when the last iterable is exhausted, filling the blanks with the `fillvalue` |

In [88]:
list(
    itertools.chain('ABC', range(2))
)

['A', 'B', 'C', 0, 1]

In [89]:
list(
    itertools.chain(enumerate('ABC'))
)

[(0, 'A'), (1, 'B'), (2, 'C')]

In [90]:
list(
    itertools.chain.from_iterable(enumerate('ABC'))
)

[0, 'A', 1, 'B', 2, 'C']

In [91]:
list(
    zip('ABC', range(5), [10, 20, 30, 40])
)

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]

In [92]:
list(
    itertools.zip_longest('ABC', range(5))
)

[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]

In [93]:
list(
    itertools.zip_longest('ABC', range(5), fillvalue='?')
)

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

In [94]:
list(
    itertools.product('ABC', range(2))
)

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [95]:
suits = 'spades hearts diamonds clubs'.split()
list(
    itertools.product('AK', suits)
)

[('A', 'spades'),
 ('A', 'hearts'),
 ('A', 'diamonds'),
 ('A', 'clubs'),
 ('K', 'spades'),
 ('K', 'hearts'),
 ('K', 'diamonds'),
 ('K', 'clubs')]

In [96]:
list(
    itertools.product('ABC')
)

[('A',), ('B',), ('C',)]

In [97]:
list(
    itertools.product('ABC', repeat=2)
)

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [98]:
list(
    itertools.product(range(2), repeat=3)
)

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [99]:
rows = itertools.product('AB', range(2), repeat=2)

In [100]:
for row in rows: print(row)

('A', 0, 'A', 0)
('A', 0, 'A', 1)
('A', 0, 'B', 0)
('A', 0, 'B', 1)
('A', 1, 'A', 0)
('A', 1, 'A', 1)
('A', 1, 'B', 0)
('A', 1, 'B', 1)
('B', 0, 'A', 0)
('B', 0, 'A', 1)
('B', 0, 'B', 0)
('B', 0, 'B', 1)
('B', 1, 'A', 0)
('B', 1, 'A', 1)
('B', 1, 'B', 0)
('B', 1, 'B', 1)


Some generator functions expand the input by yielding more than one value per input item.

| Module | Function | Description |
| :----- | :------- | :---------- |
| `itertools` | `combinations(it, out_len)` | Yields combinations of `out_len` items from the items yielded by `it` |
| `itertools` | `combinations_with_replacement(it, out_len)` | Yields combinations of `out_len` items from the items yielded by `it`, including combinations with repeated items | 
| `itertools` | `count(start=0, step=1)` | Yields numbers starting at `start`, incremented by `step`, indefinitely |
| `itertools` | `cycle(it)` | Yields items from `it`, storing a copy of each, then yields the entire sequence repeatedly, indefinitely | 
| `itertools` | `pairwise(it)` | Yields successive overlapping pairs taken from the input iterable | 
| `itertools` | `permutations(it, out_len=None)` | Yields permutations of `out_len` items from the items yielded by `it`; by default, `out_len` is `len(list(it))` | 
| `itertools` | `repeat(item, [ times ])` | Yields the item repeatedly, indefinitely unless a number of `times` is given | 

In [101]:
ct = itertools.count()
next(ct)

0

In [102]:
next(ct), next(ct), next(ct)

(1, 2, 3)

In [103]:
list(
    itertools.islice(itertools.count(1,.3), 3)
)

[1, 1.3, 1.6]

In [105]:
cy = itertools.cycle('ABC')
next(cy)

'A'

In [106]:
list(
    itertools.islice(cy, 7)
)

['B', 'C', 'A', 'B', 'C', 'A', 'B']

In [107]:
list(
    itertools.pairwise(range(7))
)

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6)]

In [108]:
rp = itertools.repeat(7)
next(rp), next(rp)

(7, 7)

In [109]:
list(
    itertools.repeat(8, 4)
)

[8, 8, 8, 8]

In [110]:
list(
    map(operator.mul, range(11), itertools.repeat(5))
)

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

The `commbinations`, `combinations_with_replacement`, and `permutations` generator functions, —together with `product`— are called the _combinatronics generators_ in the `itertools` documentation page.

In [111]:
list(
    itertools.combinations('ABC', 2)
)

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [112]:
list(
    itertools.combinations_with_replacement('ABC', 2)
)

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [113]:
list(
    itertools.permutations('ABC', 2)
)

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

In [114]:
list(
    itertools.product('ABC', repeat=2)
)

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

The last group of generator functions we’ll cover in this section are designed to yield all items in the input iterables, but rearranged in some way. Here are two functions that return multiple generators: `itertools.groupby` and `itertools.tee`. The other generator function in this group, the `reversed` built-in, is the only one covered in this section that does not accept any iterable as input, but only sequences. This makes sense: because `reversed` will yield the items from last to first, it only works with a sequence with a known length. But it avoids the cost of making a reversed copy of the sequence by yielding each item as needed. I put the `itertools.product` function together with the merging generators in Table because they all consume more than one iterable, while the generators in Table all accept at most one input iterable.

| Module | Function | Description | 
| :----- | :------- | :---------- | 
| `itertools` | `groupby(it, key=None)` | Yields 2-tuples of the form `(key, group)`, where `key` is the grouping criterion and `group` is a generator yielding items in the group | 
| (built-in) | `reversed(seq)` | Yields items from `seq` in reverse order, from last to first; `seq` must be a sequence or implement the `__reversed__` special method | 
| `itertools` | `tee(it, n=2)` | Yields a tuple of _n_ generators, each yielding the items of the input iterable independently |

In [115]:
list(
    itertools.groupby('LLLLAAGGG')
)

[('L', <itertools._grouper at 0x7fb6e0a03fa0>),
 ('A', <itertools._grouper at 0x7fb6e0a038e0>),
 ('G', <itertools._grouper at 0x7fb6e0a03c40>)]

In [117]:
for char, group in itertools.groupby('LLLLAAAGG'):
    print(char, '->', list(group))

L -> ['L', 'L', 'L', 'L']
A -> ['A', 'A', 'A']
G -> ['G', 'G']


In [119]:
animals = [
    'duck', 'eagle', 'rat', 'giraffe', 'bear',
    'bat', 'dolphin', 'shark', 'lion',
]
animals.sort(key=len)
animals

['rat', 'bat', 'duck', 'bear', 'lion', 'eagle', 'shark', 'giraffe', 'dolphin']

In [120]:
for length, group in itertools.groupby(animals, len):
    print(length, '->', list(group))

3 -> ['rat', 'bat']
4 -> ['duck', 'bear', 'lion']
5 -> ['eagle', 'shark']
7 -> ['giraffe', 'dolphin']


In [121]:
for length, group in itertools.groupby(reversed(animals), len):
    print(length, '->', list(group))

7 -> ['dolphin', 'giraffe']
5 -> ['shark', 'eagle']
4 -> ['lion', 'bear', 'duck']
3 -> ['bat', 'rat']
